In [ ]:
import json 
import notebookutils 
import sempy.fabric as fabric 
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException 

workspace_id=spark.conf.get("trident.workspace.id")
lakehouse_id=spark.conf.get("trident.lakehouse.id")

#Instantiate the client
client = fabric.FabricRestClient()

# This is the SQL endpoint I want to sync with the lakehouse, this needs to be the GUI
sqlendpoint = fabric.FabricRestClient().get(f"/v1/workspaces/{workspace_id}/lakehouses/{lakehouse_id}").json()['properties']['sqlEndpointProperties']['id']

# URI for the call 
#uri = f"v1/workspaces/{workspace_id}/sqlEndpoints/{sqlendpoint}/refreshMetadata?preview=true" #preview=true is no longer needed, the result is slightly different with preview=true
uri = f"v1/workspaces/{workspace_id}/sqlEndpoints/{sqlendpoint}/refreshMetadata" 

# This is the action, we want to take 
# payload = {} # empty payload, all tables
# Example of setting a timeout
payload = { "timeout": {"timeUnit": "Seconds", "value": "60"}  }  # Setting a timeout for the REST call

try:
    response = client.post(uri,json= payload, lro_wait = True) 
    sync_status = json.loads(response.text)
    display(sync_status)
except Exception as e: print(e)